In [41]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [42]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [43]:
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [44]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [45]:
app = Flask(__name__)

In [46]:
@app.route("/")
def welcome():
    return (
        f"Welcome to the SurfsUp in Hawaii API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start>"
        f"Enter start date"
        f"(start date must be in YYYY-MM-DD format)<br/>"
        f"/api/v1.0/<start>/<end>"
    )

In [58]:
# @app.route("/api/v1.0/precipitation")
# def precipitation():
 
session = Session(engine)
    
precipitations = session.query(Measurement.date,Measurement.prcp).all()

    # session.close()

    # results = list(np.ravel(precipitation))

precipitation_list = []
for date, prcp in precipitations:
    precipitation_dict = {date:prcp}
    # precipitation_dict["Date"] = date
    # precipitation_dict["Precipitation"] = prcp
    precipitation_list.append(precipitation_dict)
    # return jsonify(precipitation_list)
    # return jsonify([x for x in precipitations])
# print(precipitation_list)


In [48]:
# @app.route("/api/v1.0/station")
# def stations():
   
session = Session(engine)
    
stations_query = session.query(Station.station,Station.name).all()

# session.close()

# results = list(np.ravel(stations_query))
    
station_name = []
for station, name in stations_query:
    station_dict = {station:name}
    # station_dict["Station"] = station
    # station_dict["Name"] = name
    station_name.append(station_dict)
    # return jsonify(station_name)
    # return jsonify(results)
print(station_name)

[{'USC00519397': 'WAIKIKI 717.2, HI US'}, {'USC00513117': 'KANEOHE 838.1, HI US'}, {'USC00514830': 'KUALOA RANCH HEADQUARTERS 886.9, HI US'}, {'USC00517948': 'PEARL CITY, HI US'}, {'USC00518838': 'UPPER WAHIAWA 874.3, HI US'}, {'USC00519523': 'WAIMANALO EXPERIMENTAL FARM, HI US'}, {'USC00519281': 'WAIHEE 837.5, HI US'}, {'USC00511918': 'HONOLULU OBSERVATORY 702.2, HI US'}, {'USC00516128': 'MANOA LYON ARBO 785.2, HI US'}]


In [61]:
# @app.route("/api/v1.0/tobs")
# def temperature():

query_date_12_months = dt.date(2016, 8, 17) 
    
session = Session(engine)
    
temperatures = session.query(Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    order_by(Measurement.date.asc()).\
    filter(func.strftime(Measurement.date) > query_date_12_months).all()
    
    # session.close()
    
    # results = list(np.ravel(temperatures))

temperature_list = []
for tobs in temperatures:
    temperature_results = tobs
    # temperature_dict = {}
    # temperature_dict["Temperature"] = tobs
    temperature_list.append(temperature_results)
    # return jsonify(temperature)
    
    # return jsonify(results)
print(temperature_list)


[(26.7,), (26.1,), (27.2,), (26.1,), (25.6,), (25.0,), (25.0,), (26.7,), (26.7,), (23.9,), (22.8,), (25.6,), (25.0,), (25.6,), (26.7,), (26.7,), (25.6,), (25.6,), (25.6,), (22.8,), (23.3,), (26.7,), (26.1,), (25.0,), (26.7,), (24.4,), (26.1,), (23.9,), (26.1,), (25.6,), (26.1,), (25.6,), (25.6,), (24.4,), (23.3,), (25.0,), (25.6,), (26.1,), (26.1,), (25.0,), (26.7,), (25.6,), (25.6,), (25.6,), (25.0,), (26.1,), (26.1,), (26.1,), (26.1,), (23.9,), (24.4,), (22.8,), (22.2,), (21.7,), (25.0,), (26.1,), (25.6,), (26.1,), (25.0,), (26.1,), (25.0,), (25.6,), (25.6,), (25.6,), (25.6,), (25.0,), (23.3,), (23.9,), (24.4,), (22.8,), (24.4,), (23.3,), (25.0,), (24.4,), (24.4,), (23.3,), (23.9,), (23.9,), (23.9,), (23.9,), (21.7,), (17.2,), (21.1,), (20.0,), (19.4,), (25.0,), (23.3,), (25.0,), (24.4,), (24.4,), (23.9,), (24.4,), (23.9,), (22.8,), (23.9,), (22.8,), (23.9,), (23.3,), (23.9,), (23.3,), (23.9,), (22.8,), (23.9,), (22.8,), (22.8,), (23.3,), (21.1,), (22.2,), (21.1,), (19.4,), (19.4,), 

In [60]:
len(temperature_list)

357

In [73]:
min = session.query(func.min(temperature_list))
print(min)

TypeError: 'min' object is not callable

In [64]:
# @app.route("/api/v1.0/<start><br/>")
# def starting_date(start):

# """Fetch the temperatures from start date matches to last date of dataset 
#     as a path variable supplied by the user, or a 404 if not."""
     
session = Session(engine)
    
start = ("2016-08-17")
    
starting_date_list = session.query(func.min(Measurement.tobs),func.avg(Measurement.tobs),func.max(Measurement.tobs)).\
    filter(Measurement.date >= start).all()
    
session.close()
    
date_list= []
for min, max, avg in starting_date_list:
    temp_list = [min,max,avg]
    date_list.append(temp_list)

    # return jsonify({"error": f"Start date {start} not found."}), 404
   
    # results = list(np.ravel(starting_date_list))
    
    # return jsonify(date_list)
    print(date_list)

[[14.4, 23.69537037037037, 30.6]]


In [ ]:
# session.close()